# Modelo NLP: Preparación de datos.

- Usaremos la base de datos DBPedia Ontology Dataset  ( Zhang et al. https://arxiv.org/pdf/1509.01626.pdf ) 
- 12 clases, con 560,000 ejemplos de entrenamiento y  70,000 de test.
- El texto contiene el título y el resumen de cada artículo de la Wikipedia.
- El algorítmo que vamos a usar, BlazingText, espera un único fichero de texto plano donode cada linea contenga la etiqueta “_*label_*” precedida por y la frase.
- Este notebook realiza todo el procesado para obtener los ficheros para realizar el entrenamiento.

In [3]:
!wget -q https://github.com/le-scientifique/torchDatasets/raw/master/dbpedia_csv.tar.gz
!tar -xzf dbpedia_csv.tar.gz

tar: dbpedia_csv/test.csv: Cannot change ownership to uid 1000, gid 1000: Operation not permitted
tar: dbpedia_csv/classes.txt: Cannot change ownership to uid 1000, gid 1000: Operation not permitted
tar: dbpedia_csv/train.csv: Cannot change ownership to uid 1000, gid 1000: Operation not permitted
tar: dbpedia_csv/readme.txt: Cannot change ownership to uid 1000, gid 1000: Operation not permitted
tar: dbpedia_csv: Cannot change ownership to uid 1000, gid 1000: Operation not permitted
tar: Exiting with failure status due to previous errors


In [4]:
!head dbpedia_csv/train.csv -n 3

1,"E. D. Abbott Ltd"," Abbott of Farnham E D Abbott Limited was a British coachbuilding business based in Farnham Surrey trading under that name from 1929. A major part of their output was under sub-contract to motor vehicle manufacturers. Their business closed in 1972."
1,"Schwan-Stabilo"," Schwan-STABILO is a German maker of pens for writing colouring and cosmetics as well as markers and highlighters for office use. It is the world's largest manufacturer of highlighter pens Stabilo Boss."
1,"Q-workshop"," Q-workshop is a Polish company located in Poznań that specializes in designand production of polyhedral dice and dice accessories for use in various games (role-playing gamesboard games and tabletop wargames). They also run an online retail store and maintainan active forum community.Q-workshop was established in 2001 by Patryk Strzelewicz – a student from Poznań. Initiallythe company sold its products via online auction services but in 2005 a website and online store wereestablishe

In [5]:
!grep -i "automatic electric"  dbpedia_csv/train.csv 

1,"Automatic Electric"," Automatic Electric Company (AE) was the largest of the manufacturing units of the Automatic Electric Group. It was a telephone equipment supplier for independent telephone companies in North America and also had a world-wide presence. With its line of automatic telephone exchanges it was also a long-term supplier of switching equipment to the Bell System starting in 1919."
1,"Tokyo Marui"," Tokyo Marui Co. Ltd (株式会社東京マルイ Kabushiki-gaisha Tōkyō Marui) is an airsoft gun manufacturer located in Adachi Tokyo Japan. They are best known for creating the AEG(Automatic electric gun). Its main market is Japan but third-party retailers sell in Hong Kong (PRC) Taiwan (ROC) South Korea East Asia and worldwide. Such is the popularity of its guns that the company has its own center for airsoft sport called Tokyo Marui BB Sports Field."


In [6]:
!cat dbpedia_csv/classes.txt

Company
EducationalInstitution
Artist
Athlete
OfficeHolder
MeanOfTransportation
Building
NaturalPlace
Village
Animal
Plant
Album
Film
WrittenWork


In [7]:
d_label = {}
with open('dbpedia_csv/classes.txt') as f:
    for i, label in enumerate(f.readlines()):
        d_label[str(i + 1)] = label.strip()
print(d_label)

{'1': 'Company', '2': 'EducationalInstitution', '3': 'Artist', '4': 'Athlete', '5': 'OfficeHolder', '6': 'MeanOfTransportation', '7': 'Building', '8': 'NaturalPlace', '9': 'Village', '10': 'Animal', '11': 'Plant', '12': 'Album', '13': 'Film', '14': 'WrittenWork'}


In [8]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
def transform_text(row):
    cur_row = []
    label = f'__label__{d_label[row[0]]}'  # Prefix the index-ed label with __label__
    cur_row.append(label)
    cur_row.extend(nltk.word_tokenize(row[1].lower()))
    cur_row.extend(nltk.word_tokenize(row[2].lower()))
    return cur_row

In [10]:
from random import shuffle
import multiprocessing
from multiprocessing import Pool
import csv

def preprocess(input_file, output_file, keep=1):
    all_rows = []
    with open(input_file, 'r') as csvinfile:
        csv_reader = csv.reader(csvinfile, delimiter=',')
        for row in csv_reader:
            all_rows.append(row)
    shuffle(all_rows)
    all_rows = all_rows[: int(keep * len(all_rows))]
    pool = Pool(processes=multiprocessing.cpu_count())
    transformed_rows = pool.map(transform_text, all_rows)
    pool.close()
    pool.join()

    with open(output_file, 'w') as csvoutfile:
        csv_writer = csv.writer(csvoutfile, delimiter=' ', lineterminator='\n')
        csv_writer.writerows(transformed_rows)

In [11]:
%%time
preprocess('dbpedia_csv/train.csv', 'dbpedia_csv/dbpedia.train', keep=0.2)
preprocess('dbpedia_csv/test.csv', 'dbpedia_csv/dbpedia.validation')

CPU times: user 8.39 s, sys: 1.19 s, total: 9.58 s
Wall time: 1min 23s


In [12]:
!head -n 5 dbpedia_csv/dbpedia.train

__label__OfficeHolder william foushee william foushee sr. ( october 26 1749 – august 21 1824 ) was an american physician politician and socialite . after serving with distinction as a surgeon in the american revolution he entered into local virginia politics notably serving as the first mayor of richmond . he went on to become a political social and commercial leader in richmond for the next half century of his life .
__label__Plant packera cana packera cana is a species of flowering plant in the aster family known by the common name woolly groundsel . it is native to western and central north america where it can be found in a wide array of habitat types at all elevations from grassland to the alpine climates of mountain peaks . it is a perennial herb producing one erect stem from a rosette of basal leaves and a rhizome system . it generally grows up to 40 centimeters in maximum height .
__label__Animal hasora hasora the awls are a genus of skipper butterflies .
__label__Album live at